In [ ]:
#Apply SMOTE to the training set

from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42, sampling_strategy='auto')
x_train_resampled, y_train_resampled = smote.fit_resample(xtrain, ytrain)

# Perform feature scaling on the resampled training data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(x_train_resampled)

# Apply PCA for dimensional reduction on the scaled training data
pca = PCA(n_components=0.95, random_state=42)
X_train_pca = pca.fit_transform(X_train_scaled)

# Create a dataframe of PCA results for the training data
pca_columns = [f'pc{i+1}' for i in range(X_train_pca.shape[1])]
pca_df = pd.DataFrame(X_train_pca, columns=pca_columns)
pca_df

# Apply Lasso regularization on the PCA dataframe
lasso = Lasso(alpha=0.1)  # Adjust the regularization parameter alpha as per your needs
lasso.fit(pca_df, y_train_resampled)

# Filter the PCA components based on non-zero coefficients from Lasso regularization
selected_features = lasso.coef_ != 0
pca_df_selected = pca_df.loc[:, selected_features]

# Create a dataframe with the selected PCA components and target column
final_df = pd.concat([pca_df_selected, pd.Series(y_train_resampled, name='LET_IS')], axis=1)

# Replace the column names in final_df with the original dataset feature names
original_feature_names = [x.columns[i] for i, is_selected in enumerate(selected_features) if is_selected]
final_df.columns = list(original_feature_names) + ['LET_IS']

i have

In [ ]:
missing_percentages = df.isna().mean() * 100
result_df = pd.DataFrame({'column_name': missing_percentages.index, 
                          'Number of missing values': df.isna().sum(),
                          'percent_missing': missing_percentages.values})
print(result_df)

#Get the columns that have more than 30% missing values
cols_with_more_than_30percent_missing = result_df[result_df['percent_missing'] > 30]['column_name']

#Print the result
print(cols_with_more_than_30percent_missing)

df.drop(columns=cols_with_more_than_30percent_missing, inplace=True)
df

# Drop Columns of Myocardial Infraction Outcomes
columns_to_drop = ['FIBR_PREDS', 'PREDS_TAH', 'JELUD_TAH', 'FIBR_JELUD', 'A_V_BLOK', 'OTEK_LANC', 'RAZRIV', 'DRESSLER', 'ZSN', 'REC_IM', 'P_IM_STEN']
df = df.drop(columns_to_drop, axis=1)

# missing values per row (selected column threshold: 30%)
row_list = []
for i in range(df.shape[0]):
    n_miss = df.iloc[i].isnull().sum()
    perc = n_miss / df.shape[1] * 100
    # print('> %d, Missing: %d (%.1f%%)' % (i, n_miss, perc))
    if perc >= 30:
        row_list.append(i)

df.drop(row_list, inplace=True)
df

# Create an instance of SimpleImputer with strategy='most_frequent'
imputer = SimpleImputer(strategy='most_frequent')
# Fill missing values in your DataFrame with the mode
x_filled = imputer.fit_transform(df)
# Convert the filled array back to a DataFrame
df = pd.DataFrame(x_filled, columns=df.columns)
df



In [ ]:
# Create a dataframe with the selected PCA components and target column
final_df = pd.DataFrame(X_pca_selected, columns=[f'pc{i+1}' for i in range(X_pca_selected.shape[1])])
final_df["LET_IS"] = y_resampled